# Qutrit Quantum Kernel with Classical Support Vector Machine (SVM) for Iris Dataset

In this notebook, is the implementation of a quantum kernel combined with a classical SVM approach to tackle the Iris dataset. The Iris dataset is a well-known multi-class classification problem, consisting of samples from three different classes of iris flowers.

The implementation process involves data preprocessing, quantum feature mapping, classical SVM training, and evaluation. In this particular architecture two qutrits are going to be used instead of only a single qutrit as in the binary classification problems.

I'll import the necessary libraries at the beginning of the notebook to facilitate a smooth and efficient workflow. Let's proceed with building and evaluating the Qutrit Quantum Kernel combined with the classical SVM on the Iris dataset.

In [1]:
import numpy as np
import torch
from scipy.linalg import expm
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from matplotlib.colors import ListedColormap
plt.style.use('seaborn-v0_8')

#### Qutrits

Defining the qutrits that have three possible states, denoted as |0⟩, |1⟩, and |2⟩. 

In [2]:
# Define the qutrit states as column vectors
q0 = np.array([[1], [0], [0]])
q1 = np.array([[0], [1], [0]])
q2 = np.array([[0], [0], [1]])

#### Gell-Mann Matrices

The generators of a qutrit are the operators that generate transformations on the qutrit states under some symmetry group. One common set of generators for a qutrit are the Gell-Mann matrices, which are a set of 8 Hermitian operators that span the space of 3x3 complex matrices:

$gm1 = |0⟩⟨1| + |1⟩⟨0| \\
gm2 = -i(|0⟩⟨1| - |1⟩⟨0|) \\
gm3 = |0⟩⟨0| - |1⟩⟨1| \\
gm4 = |0⟩⟨2| + |2⟩⟨0| \\
gm5 = -i(|0⟩⟨2| - |2⟩⟨0|) \\
gm6 = |1⟩⟨2| + |2⟩⟨1| \\
gm7 = -i(|1⟩⟨2| - |2⟩⟨1|) \\
gm8 = 1/√3 (|0⟩⟨0| + |1⟩⟨1| - 2|2⟩⟨2|)$

These generators satisfy the commutation relations of the SU(3) Lie algebra, which is the symmetry group of the qutrit. The Gell-Mann matrices can be used to construct any unitary transformation on the qutrit, making them a useful tool for analyzing the behavior of qutrit systems in quantum mechanics.

In [3]:
# Define the Gell-Mann matrices
gm1 = np.kron(q0, q1.T) + np.kron(q1, q0.T)
gm2 = -1j * (np.kron(q0, q1.T) - np.kron(q1, q0.T))
gm3 = np.kron(q0, q0.T) - np.kron(q1, q1.T)
gm4 = np.kron(q0, q2.T) + np.kron(q2, q0.T)
gm5 = -1j * (np.kron(q0, q2.T) - np.kron(q2, q0.T))
gm6 = np.kron(q1, q2.T) + np.kron(q2, q1.T)
gm7 = -1j * (np.kron(q1, q2.T) - np.kron(q2, q1.T))
gm8 = 1/np.sqrt(3) * (np.kron(q0, q0.T) + np.kron(q1, q1.T) - 2*np.kron(q2, q2.T))

# Collect the Glenn-Mann matrices in a list
generators = [gm1, gm2, gm3, gm4, gm5, gm6, gm7, gm8]

# Print Glenn-Mann 8
print(gm8)

[[ 0.57735027  0.          0.        ]
 [ 0.          0.57735027  0.        ]
 [ 0.          0.         -1.15470054]]


Then defining the Hadamard operator for qutrits.

In [4]:
# Define the Hadamard operator for qutrits
H = (1/np.sqrt(3)) * np.array([[1, 1, 1], [1, np.exp(2j*np.pi/3), np.exp(-2j*np.pi/3)], [1, np.exp(-2j*np.pi/3), np.exp(2j*np.pi/3)]])

# Print the Hadamard operator
print("Hadamard operator for qutrits:")
print(H)

Hadamard operator for qutrits:
[[ 0.57735027+0.j   0.57735027+0.j   0.57735027+0.j ]
 [ 0.57735027+0.j  -0.28867513+0.5j -0.28867513-0.5j]
 [ 0.57735027+0.j  -0.28867513-0.5j -0.28867513+0.5j]]


### Quantum Kernel

To construct the quantum kernel, I'll use custom functions that leverage the principles of quantum computing to transform the input data into a high-dimensional Hilbert space. The quantum kernel captures complex and non-linear relationships between data points, making it advantageous for certain types of datasets, such as those with intricate decision boundaries.

In [5]:
# Encoding four features on a qutrit
def encoding(vector):

    sum = 0
    for i in range(4):
        sum = sum + (1j * vector[i] * generators[i]) 

    return np.dot(expm(sum), np.dot(H,q0))

Testing to see if the encoding works with a simple vector.

In [9]:
vector = np.array([6.34, 22.11, 2,3], dtype=complex)
vector2 = np.array([-6.34, 22.11, 1, 4], dtype=complex)

print(encoding(vector))
print(encoding(vector2))
print("multi")
print(np.real(np.dot(encoding(vector).conj().T, encoding(vector))**2))

[[-0.68960068-2.82036151e-01j]
 [ 0.35344573-1.85741072e-01j]
 [ 0.53430595-2.00022498e-04j]]
[[-0.65299607+0.02265479j]
 [ 0.47112451+0.07255427j]
 [ 0.5880902 -0.00322283j]]
multi
[[1.]]


Then defining the LZZ2 gate which is going to be used as an entanglement operator.

In [6]:
LZ2 = gm3 + np.sqrt(3)*gm8
LZZ2 = np.kron(LZ2, LZ2)
print(LZ2)
print(LZZ2)

[[ 2.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -2.]]
[[ 4.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -4.  0.  0. -0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0. -0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.  0. -4. -0. -0.]
 [ 0.  0.  0.  0.  0.  0. -0. -0. -0.]
 [ 0.  0. -0.  0.  0. -0. -0. -0.  4.]]


Then defining the kernel function, which quantifies the similarity or correlation between the quantum states of x1 and x2 and serves as a critical component in quantum machine learning algorithms like Support Vector Machines (SVM) for classification tasks, where the quantum advantage lies in capturing non-linear relationships between data points in high-dimensional quantum spaces.

In [7]:

def kernel(x1, x2):
    """The quantum kernel."""
    qutrit_1 = encoding(x1)
    qutrit_2 = encoding(x1)

    # Entanglement gate
    entangle_gate = 1j*LZZ2

    # Applying entanglement between the three qutrits
    qutrit_1x2 = np.kron(qutrit_1, qutrit_2)
    kron1 = np.dot(expm(entangle_gate), qutrit_1x2)

    qutrit_1 = encoding(x2)
    qutrit_2 = encoding(x2)

    # Applying entanglement between the three qutrits
    qutrit_1x2 = np.kron(qutrit_1, qutrit_2)
    kron2 = np.dot(expm(entangle_gate), qutrit_1x2)

    return np.real(np.dot(kron1.conj().T, kron2)**2)[0][0]

In [10]:
print(kernel(vector, vector2))

0.6505781279663496


In [11]:
x = torch.tensor(kernel(vector, vector), requires_grad=True)
print(x)

tensor(1.0000, dtype=torch.float64, requires_grad=True)


In [12]:

def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

### Iris dataset

Each sample in the Iris dataset is characterized by four features: sepal length, sepal width, petal length, and petal width, all measured in centimeters. The goal of this classification task is to predict the species of the iris flower based on these four features. In the dataset ingestion step, we'll load the Iris dataset using scikit-learn.

In [13]:
X, y = load_iris(return_X_y=True)

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y_scaled = 2 * (y - 0.5)

print('Shape of X:', X_scaled.shape)
print('Shape of y:', y_scaled.shape)
print('x[0] feature example: ', X_scaled[0])
print('y[0]: ', y_scaled[142])

Shape of X: (150, 4)
Shape of y: (150,)
x[0] feature example:  [-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
y[0]:  3.0


To split the dataset into a training set and a test set into a 80-20 train-test split with equal representation of the two classes in both sets, I'll use the train_test_split function from sklearn.model_selection module with the stratify parameter set to the target variable y.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, stratify=y_scaled, random_state=42)

# Use np.sum to count the number of instances of each class in both sets
print("Class counts in training set:", np.sum(y_train.shape))
print("Class counts in test set:", np.sum(y_test.shape))


Class counts in training set: 120
Class counts in test set: 30


In [15]:
print(kernel(X_train[0], X_train[0]))

1.0000000000000013


### Classical SVM

For the classical SVM component, I'll rely on the popular scikit-learn library, a powerful toolset for machine learning in Python. The SVM aims to find the optimal hyperplane that separates data points of different classes while maximizing the margin between them. By combining the quantum kernel with the classical SVM, we can handle multi-class classification tasks effectively.

In [16]:

svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [17]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

0.9

### Evaluation

In [18]:
# To be printed better
y_pred = predictions
y_true = y_test
accuracy = accuracy_score(y_true, y_pred)* 100
f1 = f1_score(y_true, y_pred, average='macro')* 100
precision = precision_score(y_true, y_pred, average='macro')* 100
recall = recall_score(y_true, y_pred, average='macro')* 100

# Print the results
print("Evaluation Results")
print("_____________________________________________")
print(
            f"\nRecall: {recall:.2f}%"
            f"\nPrecision: {precision:.2f}%"
            f"\nAccuracy: {accuracy:.2f}%"
            f"\nMacro Averaged F1-score: {f1:.2f}%"
            )
print("_____________________________________________")

Evaluation Results
_____________________________________________

Recall: 90.00%
Precision: 92.31%
Accuracy: 90.00%
Macro Averaged F1-score: 89.77%
_____________________________________________
